In [1]:
from ase_interface import ANIENS
from ase_interface import ensemblemolecule

import pyNeuroChem as pync
import pyaniasetools as pya
import hdnntools as hdt

import numpy as np
import  ase

import time

import os
os.environ["OMP_NUM_THREADS"] = "2"

import  ase
from ase.md.langevin import Langevin
from ase.md.verlet import VelocityVerlet
from ase.io.trajectory import Trajectory
from ase import units

from ase.optimize.fire import FIRE as QuasiNewton

from ase.md.nvtberendsen import NVTBerendsen
from ase.md import MDLogger

from ase.io import read, write

from ase.parallel import world

In [2]:
dir = '/home/jsmith48/scratch/MD_TEST/2luf/'

# Molecule file
molfile = dir + '2luf_solv.pdb'

# Dynamics file
xyzfile = dir + 'mdcrd.xyz'

# Trajectory file
trajfile = dir + 'traj.dat'

# Optimized structure out:
optfile = dir + 'optmol.xyz'

T = 300.0 # Temperature
dt = 0.25
C = 0.1 # Optimization convergence
steps = 1000

ntdir = '/home/jsmith48/scratch/ANI-2x_retrain/model3/'
cns = ntdir + 'rHCNOSFCl-4.6R_16-3.1A_a4-8.params'
sae = ntdir + 'sae_linfit.dat'
nnf = ntdir + 'train'
Nn = 5


#ntdir = '/home/jsmith48/scratch/ANI-1x_retrain/train_ens/'
#cns = ntdir + 'rHCNO-4.6R_16-3.1A_a4-8.params'
#sae = ntdir + 'sae_linfit.dat'
#nnf = ntdir + 'train'
#Nn = 5

In [3]:
# Load molecule
mol = read(molfile)
L = 40.0
mol.set_cell(([[L, 0, 0],
               [0, L, 0],
               [0, 0, L]]))

mol.set_pbc((True, True, True))

# Set NC
aens = ensemblemolecule(cns, sae, nnf, Nn, 4)

# Set ANI calculator
mol.set_calculator(ANIENS(aens,sdmx=20000000.0))

s_time = time.time()
E = mol.get_potential_energy()
F = mol.get_forces()
print('Total time:',time.time()-s_time)

spc = mol.get_chemical_symbols()
print(len(spc))

Total time: 0.25377988815307617
6732


In [4]:
# Optimize molecule
start_time = time.time()
dyn = QuasiNewton(mol)
dyn.run(fmax=C)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

# Save optimized mol
spc = mol.get_chemical_symbols()
pos = mol.get_positions(wrap=False).reshape(1,len(spc),3)


hdt.writexyzfile(optfile, pos, spc)

      Step     Time          Energy         fmax
FIRE:    0 10:16:50 -4656542.539497        4.2935
FIRE:    1 10:16:50 -4656572.131703        4.1219
FIRE:    2 10:16:50 -4656599.350924        3.9608
FIRE:    3 10:16:50 -4656624.244125        3.8146
FIRE:    4 10:16:50 -4656646.877534        3.6781
FIRE:    5 10:16:50 -4656667.336922        3.5532
FIRE:    6 10:16:50 -4656685.735753        3.4384
FIRE:    7 10:16:50 -4656702.221486        3.3225
FIRE:    8 10:16:50 -4656717.000343        3.2360
FIRE:    9 10:16:51 -4656730.334066        3.1792
FIRE:   10 10:16:51 -4656742.534080        3.1368
FIRE:   11 10:16:51 -4656753.946372        3.1010
FIRE:   12 10:16:51 -4656764.917538        3.0635
FIRE:   13 10:16:51 -4656775.753960        3.0201
FIRE:   14 10:16:51 -4656786.679148        2.9712
FIRE:   15 10:16:51 -4656797.807916        2.9148
FIRE:   16 10:16:51 -4656809.146400        2.8466
FIRE:   17 10:16:51 -4656820.616711        2.7805
FIRE:   18 10:16:51 -4656832.093954        2.7128
F

FIRE:  163 10:17:01 -4657619.090129        0.7115
FIRE:  164 10:17:01 -4657621.682154        0.7044
FIRE:  165 10:17:01 -4657624.253625        0.6960
FIRE:  166 10:17:01 -4657626.804836        0.6871
FIRE:  167 10:17:01 -4657629.335967        0.6769
FIRE:  168 10:17:01 -4657631.847219        0.6661
FIRE:  169 10:17:01 -4657634.338726        0.6562
FIRE:  170 10:17:01 -4657636.810688        0.6596
FIRE:  171 10:17:01 -4657639.263254        0.6615
FIRE:  172 10:17:01 -4657641.696574        0.6606
FIRE:  173 10:17:01 -4657644.110944        0.6598
FIRE:  174 10:17:01 -4657646.506770        0.6589
FIRE:  175 10:17:01 -4657648.884405        0.6567
FIRE:  176 10:17:02 -4657651.244079        0.6540
FIRE:  177 10:17:02 -4657653.586100        0.6512
FIRE:  178 10:17:02 -4657655.910677        0.6485
FIRE:  179 10:17:02 -4657658.218122        0.6464
FIRE:  180 10:17:02 -4657660.508510        0.6424
FIRE:  181 10:17:02 -4657662.781866        0.6384
FIRE:  182 10:17:02 -4657665.038343        0.6356


FIRE:  327 10:17:11 -4657856.257742        0.8790
FIRE:  328 10:17:11 -4657856.850763        0.8934
FIRE:  329 10:17:11 -4657857.436745        0.9067
FIRE:  330 10:17:12 -4657858.015817        0.9189
FIRE:  331 10:17:12 -4657858.587981        0.9302
FIRE:  332 10:17:12 -4657859.153420        0.9403
FIRE:  333 10:17:12 -4657859.712161        0.9493
FIRE:  334 10:17:12 -4657860.264311        0.9564
FIRE:  335 10:17:12 -4657860.809919        0.9621
FIRE:  336 10:17:12 -4657861.349087        0.9664
FIRE:  337 10:17:12 -4657861.881743        0.9687
FIRE:  338 10:17:12 -4657862.408044        0.9698
FIRE:  339 10:17:12 -4657862.928174        0.9692
FIRE:  340 10:17:12 -4657863.442116        0.9668
FIRE:  341 10:17:12 -4657863.949923        0.9632
FIRE:  342 10:17:12 -4657864.451369        0.9586
FIRE:  343 10:17:13 -4657864.946385        0.9531
FIRE:  344 10:17:13 -4657865.435015        0.9466
FIRE:  345 10:17:13 -4657865.917283        0.9393
FIRE:  346 10:17:13 -4657866.393295        0.9313


FIRE:  491 10:17:22 -4657897.176518        1.1073
FIRE:  492 10:17:22 -4657897.303301        1.1044
FIRE:  493 10:17:22 -4657897.429299        1.1006
FIRE:  494 10:17:22 -4657897.554440        1.0958
FIRE:  495 10:17:22 -4657897.678781        1.0897
FIRE:  496 10:17:22 -4657897.802227        1.0826
FIRE:  497 10:17:22 -4657897.924778        1.0741
FIRE:  498 10:17:22 -4657898.046481        1.0646
FIRE:  499 10:17:22 -4657898.167412        1.0547
FIRE:  500 10:17:23 -4657898.287495        1.0442
FIRE:  501 10:17:23 -4657898.406862        1.0329
FIRE:  502 10:17:23 -4657898.525524        1.0206
FIRE:  503 10:17:23 -4657898.643578        1.0062
FIRE:  504 10:17:23 -4657898.761072        0.9914
FIRE:  505 10:17:23 -4657898.878234        0.9767
FIRE:  506 10:17:23 -4657898.995155        0.9620
FIRE:  507 10:17:23 -4657899.111928        0.9475
FIRE:  508 10:17:23 -4657899.228780        0.9331
FIRE:  509 10:17:23 -4657899.345714        0.9362
FIRE:  510 10:17:23 -4657899.462758        0.9523


FIRE:  655 10:17:32 -4657918.430648        1.1212
FIRE:  656 10:17:32 -4657918.564740        1.1168
FIRE:  657 10:17:32 -4657918.698591        1.1115
FIRE:  658 10:17:32 -4657918.832242        1.1050
FIRE:  659 10:17:32 -4657918.965657        1.0977
FIRE:  660 10:17:32 -4657919.099004        1.0898
FIRE:  661 10:17:33 -4657919.232200        1.0812
FIRE:  662 10:17:33 -4657919.365262        1.0722
FIRE:  663 10:17:33 -4657919.498030        1.0630
FIRE:  664 10:17:33 -4657919.630621        1.0596
FIRE:  665 10:17:33 -4657919.762882        1.0816
FIRE:  666 10:17:33 -4657919.894729        1.1018
FIRE:  667 10:17:33 -4657920.026176        1.1193
FIRE:  668 10:17:33 -4657920.157415        1.1339
FIRE:  669 10:17:33 -4657920.288487        1.1452
FIRE:  670 10:17:33 -4657920.419447        1.1531
FIRE:  671 10:17:33 -4657920.550471        1.1573
FIRE:  672 10:17:33 -4657920.681540        1.1578
FIRE:  673 10:17:33 -4657920.812704        1.1546
FIRE:  674 10:17:33 -4657920.944041        1.1474


...

FIRE:  983 10:17:52 -4658075.850677        0.3306
FIRE:  984 10:17:52 -4658077.084055        0.3281
FIRE:  985 10:17:52 -4658078.296063        0.3281
FIRE:  986 10:17:52 -4658079.486993        0.3265
FIRE:  987 10:17:53 -4658080.657303        0.3250
FIRE:  988 10:17:53 -4658081.807631        0.3234
FIRE:  989 10:17:53 -4658082.938480        0.3211
FIRE:  990 10:17:53 -4658084.050341        0.3143
FIRE:  991 10:17:53 -4658085.143780        0.3136
FIRE:  992 10:17:53 -4658086.219348        0.3122
FIRE:  993 10:17:53 -4658087.277392        0.3095
FIRE:  994 10:17:53 -4658088.318444        0.3062
FIRE:  995 10:17:53 -4658089.342949        0.3020
FIRE:  996 10:17:53 -4658090.351280        0.2963
FIRE:  997 10:17:53 -4658091.343748        0.2896
FIRE:  998 10:17:53 -4658092.320646        0.2812
FIRE:  999 10:17:53 -4658093.282387        0.2725
FIRE:  1000 10:17:53 -4658094.229265        0.2637
FIRE:  1001 10:17:53 -4658095.161640        0.2555
FIRE:  1002 10:17:53 -4658096.079786        0.25

FIRE:  1144 10:18:02 -4658162.582979        0.2540
FIRE:  1145 10:18:02 -4658162.835266        0.2510
FIRE:  1146 10:18:02 -4658163.085560        0.2468
FIRE:  1147 10:18:03 -4658163.333761        0.2490
FIRE:  1148 10:18:03 -4658163.579841        0.2510
FIRE:  1149 10:18:03 -4658163.823839        0.2517
FIRE:  1150 10:18:03 -4658164.065670        0.2511
FIRE:  1151 10:18:03 -4658164.305290        0.2497
FIRE:  1152 10:18:03 -4658164.542720        0.2478
FIRE:  1153 10:18:03 -4658164.777985        0.2631
FIRE:  1154 10:18:03 -4658165.011111        0.2759
FIRE:  1155 10:18:03 -4658165.242122        0.2860
FIRE:  1156 10:18:03 -4658165.471042        0.2935
FIRE:  1157 10:18:03 -4658165.697828        0.2984
FIRE:  1158 10:18:03 -4658165.922486        0.3009
FIRE:  1159 10:18:03 -4658166.144981        0.3010
FIRE:  1160 10:18:03 -4658166.365220        0.2987
FIRE:  1161 10:18:03 -4658166.583207        0.2943
FIRE:  1162 10:18:03 -4658166.798838        0.2879
FIRE:  1163 10:18:03 -4658167.0

FIRE:  1305 10:18:12 -4658187.301372        0.3405
FIRE:  1306 10:18:12 -4658187.406724        0.3411
FIRE:  1307 10:18:12 -4658187.512408        0.3412
FIRE:  1308 10:18:13 -4658187.618451        0.3406
FIRE:  1309 10:18:13 -4658187.724715        0.3395
FIRE:  1310 10:18:13 -4658187.831259        0.3381
FIRE:  1311 10:18:13 -4658187.937891        0.3381
FIRE:  1312 10:18:13 -4658188.044626        0.3469
FIRE:  1313 10:18:13 -4658188.151439        0.3551
FIRE:  1314 10:18:13 -4658188.258305        0.3623
FIRE:  1315 10:18:13 -4658188.365176        0.3679
FIRE:  1316 10:18:13 -4658188.472198        0.3715
FIRE:  1317 10:18:13 -4658188.579347        0.3728
FIRE:  1318 10:18:13 -4658188.686528        0.3715
FIRE:  1319 10:18:13 -4658188.793851        0.3675
FIRE:  1320 10:18:13 -4658188.901325        0.3616
FIRE:  1321 10:18:13 -4658189.008882        0.3544
FIRE:  1322 10:18:13 -4658189.116401        0.3464
FIRE:  1323 10:18:14 -4658189.223644        0.3382
FIRE:  1324 10:18:14 -4658189.3

FIRE:  1466 10:18:23 -4658200.331383        0.3636
FIRE:  1467 10:18:23 -4658200.384627        0.3713
FIRE:  1468 10:18:23 -4658200.437943        0.3774
FIRE:  1469 10:18:23 -4658200.491490        0.3817
FIRE:  1470 10:18:23 -4658200.545374        0.3840
FIRE:  1471 10:18:23 -4658200.599582        0.3902
FIRE:  1472 10:18:23 -4658200.654048        0.3977
FIRE:  1473 10:18:23 -4658200.708880        0.4054
FIRE:  1474 10:18:23 -4658200.764022        0.4134
FIRE:  1475 10:18:23 -4658200.819317        0.4218
FIRE:  1476 10:18:24 -4658200.874861        0.4304
FIRE:  1477 10:18:24 -4658200.930538        0.4391
FIRE:  1478 10:18:24 -4658200.986412        0.4474
FIRE:  1479 10:18:24 -4658201.042493        0.4550
FIRE:  1480 10:18:24 -4658201.098903        0.4620
FIRE:  1481 10:18:24 -4658201.155724        0.4681
FIRE:  1482 10:18:24 -4658201.212894        0.4734
FIRE:  1483 10:18:24 -4658201.270300        0.4776
FIRE:  1484 10:18:24 -4658201.327928        0.4808
FIRE:  1485 10:18:24 -4658201.3

FIRE:  1627 10:18:33 -4658208.967759        0.4629
FIRE:  1628 10:18:33 -4658209.004813        0.4739
FIRE:  1629 10:18:33 -4658209.041904        0.4836
FIRE:  1630 10:18:33 -4658209.079342        0.4919
FIRE:  1631 10:18:33 -4658209.117230        0.4984
FIRE:  1632 10:18:33 -4658209.155686        0.5032
FIRE:  1633 10:18:33 -4658209.194752        0.5064
FIRE:  1634 10:18:33 -4658209.234425        0.5083
FIRE:  1635 10:18:33 -4658209.274837        0.5086
FIRE:  1636 10:18:33 -4658209.315848        0.5076
FIRE:  1637 10:18:33 -4658209.357417        0.5054
FIRE:  1638 10:18:33 -4658209.399469        0.5016
FIRE:  1639 10:18:33 -4658209.441998        0.4967
FIRE:  1640 10:18:34 -4658209.484946        0.4908
FIRE:  1641 10:18:34 -4658209.528300        0.4932
FIRE:  1642 10:18:34 -4658209.572031        0.5006
FIRE:  1643 10:18:34 -4658209.616131        0.5071
FIRE:  1644 10:18:34 -4658209.660597        0.5128
FIRE:  1645 10:18:34 -4658209.705316        0.5175
FIRE:  1646 10:18:34 -4658209.7

In [5]:
# We want to run MD with constant energy using the VelocityVerlet algorithm.
#dyn = NVTBerendsen(mol, 1 * units.fs, 100.0, taut=1.0*1000*units.fs)
# coefficient to 0.04 atomic units.
dyn = Langevin(mol, dt * units.fs, T * units.kB, 0.04)

In [6]:
# Open MD output
mdcrd = open(xyzfile,'w')

# Open MD output
traj = open(trajfile,'w')

# Define the printer
def storeenergy(a=mol, d=dyn, b=mdcrd, t=traj):  # store a reference to atoms in the definition.
    """Function to print the potential, kinetic and total energy."""
    epot = a.get_potential_energy() / len(a)
    ekin = a.get_kinetic_energy() / len(a)

    stddev =  hdt.evtokcal*a.calc.stddev

    t.write(str(d.get_number_of_steps()) + ' ' + str(ekin / (1.5 * units.kB)) + ' ' + str(epot) + ' ' + str(ekin) + ' ' + str(epot+ekin) + ' ' + str(stddev) + '\n')
    b.write(str(len(a)) + '\n' + str(ekin / (1.5 * units.kB)) + ' Step: ' + str(d.get_number_of_steps()) + '\n')
    c = a.get_positions(wrap=True)
    for j, i in zip(a, c):
        b.write(str(j.symbol) + ' ' + str(i[0]) + ' ' + str(i[1]) + ' ' + str(i[2]) + '\n')

    print('Step: %d Energy per atom: Epot = %.3f  Ekin = %.3f (T=%.3fK)  '
          'Etot = %.6f' % (d.get_number_of_steps(), epot, ekin, ekin / (1.5 * units.kB), epot + ekin))
    
# Set the printer
dyn.attach(storeenergy, interval=10)

In [7]:
start_time = time.time()
dyn.set_temperature(300. * units.kB)
dyn.run(steps)
print('[ANI Total time:', time.time() - start_time, 'seconds]')

Step: 10 Energy per atom: Epot = -691.955  Ekin = 0.001 (T=5.028K)  Etot = -691.954427
Step: 20 Energy per atom: Epot = -691.955  Ekin = 0.001 (T=9.086K)  Etot = -691.953684
Step: 30 Energy per atom: Epot = -691.955  Ekin = 0.002 (T=12.318K)  Etot = -691.952948
Step: 40 Energy per atom: Epot = -691.954  Ekin = 0.002 (T=15.783K)  Etot = -691.952194
Step: 50 Energy per atom: Epot = -691.954  Ekin = 0.002 (T=18.541K)  Etot = -691.951488
Step: 60 Energy per atom: Epot = -691.953  Ekin = 0.003 (T=21.049K)  Etot = -691.950778
Step: 70 Energy per atom: Epot = -691.953  Ekin = 0.003 (T=23.454K)  Etot = -691.950114
Step: 80 Energy per atom: Epot = -691.953  Ekin = 0.003 (T=26.753K)  Etot = -691.949406
Step: 90 Energy per atom: Epot = -691.953  Ekin = 0.004 (T=29.664K)  Etot = -691.948721
Step: 100 Energy per atom: Epot = -691.952  Ekin = 0.004 (T=32.143K)  Etot = -691.948055
Step: 110 Energy per atom: Epot = -691.952  Ekin = 0.004 (T=34.633K)  Etot = -691.947412
Step: 120 Energy per atom: Epot 

Step: 930 Energy per atom: Epot = -691.933  Ekin = 0.024 (T=183.735K)  Etot = -691.909749
Step: 940 Energy per atom: Epot = -691.933  Ekin = 0.024 (T=184.302K)  Etot = -691.909479
Step: 950 Energy per atom: Epot = -691.933  Ekin = 0.024 (T=187.083K)  Etot = -691.909139
Step: 960 Energy per atom: Epot = -691.933  Ekin = 0.024 (T=186.946K)  Etot = -691.908829
Step: 970 Energy per atom: Epot = -691.933  Ekin = 0.024 (T=187.101K)  Etot = -691.908558
Step: 980 Energy per atom: Epot = -691.933  Ekin = 0.025 (T=190.007K)  Etot = -691.908195
Step: 990 Energy per atom: Epot = -691.932  Ekin = 0.024 (T=189.217K)  Etot = -691.908024
Step: 1000 Energy per atom: Epot = -691.932  Ekin = 0.025 (T=190.283K)  Etot = -691.907686
[ANI Total time: 66.1646945476532 seconds]


In [8]:
dyn.run(steps)

Step: 1010 Energy per atom: Epot = -691.932  Ekin = 0.025 (T=191.263K)  Etot = -691.907439
Step: 1020 Energy per atom: Epot = -691.932  Ekin = 0.025 (T=193.212K)  Etot = -691.907117
Step: 1030 Energy per atom: Epot = -691.932  Ekin = 0.025 (T=194.146K)  Etot = -691.906910
Step: 1040 Energy per atom: Epot = -691.932  Ekin = 0.025 (T=195.994K)  Etot = -691.906647
Step: 1050 Energy per atom: Epot = -691.932  Ekin = 0.026 (T=197.779K)  Etot = -691.906356
Step: 1060 Energy per atom: Epot = -691.932  Ekin = 0.026 (T=197.986K)  Etot = -691.906114
Step: 1070 Energy per atom: Epot = -691.931  Ekin = 0.026 (T=197.625K)  Etot = -691.905954
Step: 1080 Energy per atom: Epot = -691.931  Ekin = 0.026 (T=197.694K)  Etot = -691.905722
Step: 1090 Energy per atom: Epot = -691.931  Ekin = 0.026 (T=200.204K)  Etot = -691.905478
Step: 1100 Energy per atom: Epot = -691.931  Ekin = 0.026 (T=199.963K)  Etot = -691.905175
Step: 1110 Energy per atom: Epot = -691.931  Ekin = 0.026 (T=204.410K)  Etot = -691.904842

KeyboardInterrupt: 